# Learning Associations : Grocery Dataset#

##  1. Importing Libraries ##

In [1]:
import numpy as np # linear algebra
import pandas as pd # for pre - processing
import matplotlib.pyplot as plt # for Data - Visualization
from scipy.special import comb # The number of combinations of N things taken K at a time; "N choose K"
from itertools import combinations, permutations # to form a "iterator algebra"
import pyfpgrowth
import url

## 1.1. Installing Libraries for Algorithms ##

In [2]:
#! pip install apyori # Apriori algorith to find the Associations of the Grocery Data
#! pip install pyfpgrowth
#! pip install url

## 2. Load Grocery File ##

In [152]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_10b8e41eca1446128325a8a246b62cfc = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ZN1aRkQJOcPzNQiCLwUm7IXqjH8SJBqtAhdvnmQQTg43',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_10b8e41eca1446128325a8a246b62cfc.get_object(Bucket='learningassociationsgroceries-donotdelete-pr-bmc4wwzdn1kprb',Key='groceries - groceries.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head(5)


,Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,4,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Observations from the Dataset #

## The groceries - groceries data consists of Transactions identifiers and comprises of 32 itemlist ##

In [4]:
def apyori(df, minimum_support=0.1, confidence=0.22):
    df_values = df.values.astype(str)
    index, counts = np.unique(df_values,return_counts=True)
    df_item = pd.DataFrame(zip(index, counts), columns = ['product', 'frequency'])
    df_item.drop(df_item[(df_item['product'] == 'nan' )|(df_item['product'] == 'None' )].index, inplace=True)
    df_item.sort_values(by='frequency', ascending=False, inplace=True)
    df_item.reset_index(drop=True, inplace=True)
    df_item_frequent = df_item[df_item['frequency']>= minimum_support*len(df)]
    df_itemset_frequency = pd.DataFrame(columns=['itemset', 'frequency'])
    for i in range(1, len(df_item_frequent)+1):
        comb = list(combinations(df_item_frequent['product'].values, i) )
        for w in comb:
            count = 0 
            for instance in df_values:
                if all(elem in instance  for elem in w):
                    count = count +1
            if count >= (minimum_support*len(df)/2):#tirar /2
                df_itemset_frequency = df_itemset_frequency.append({'itemset':w, 'frequency':count}, ignore_index=True)
    df_itemset_frequency.sort_values(by='frequency', inplace=True, ascending=False)
    reliability = pd.DataFrame(columns=['rule', 'frequency', 'reliability'])
    for w in df_itemset_frequency['itemset'].values:
        w_p = list(permutations(w,len(w)))
        for j in w_p:
            #print (len(j[0]))

            p_uni = []
            for i in range(len(j)):

                count = 0 
                for instance in df_values:
                    if all(elem in instance  for elem in j[i:]):
                        count = count +1
                p_uni.append(count/len(df))

            if len(j) != 1:
                a = p_uni[-2]/p_uni[-1]

                for i in range(len(p_uni)-2):
                    a = p_uni[-i-3]/a
                j = list(j)
                j.reverse()
                reliability = reliability.append({'rule':j, 'frequency':p_uni[0], 'reliability':a}, ignore_index=True)
            else:
                reliability = reliability.append({'rule':j, 'frequency':p_uni[0], 'reliability':p_uni[0]}, ignore_index=True)
    reliability.sort_values(by='frequency', ascending=False)
    return reliability[reliability['reliability']>=confidence]
apyori(df.drop(columns='Item(s)'))

,rule,frequency,reliability
0,"(whole milk,)",0.255516,0.255516
8,"[other vegetables, whole milk]",0.074835,0.386758
9,"[whole milk, other vegetables]",0.074835,0.292877
10,"[rolls/buns, whole milk]",0.056634,0.307905
11,"[whole milk, rolls/buns]",0.056634,0.221647
12,"[yogurt, whole milk]",0.056024,0.401603


In [10]:
df1 = df.drop(columns='Item(s)')
df1.head(5)

,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,Item 10,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
df2 = df1.iloc[:, 0 : 33]
df2.head(5)

,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,Item 10,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
patterns = pyfpgrowth.find_frequent_patterns(df2, 10)

In [92]:
rules = pyfpgrowth. generate_association_rules(patterns,0.8)

In [93]:
#!pip install transaction

In [94]:
#! pip install mlxtend

In [135]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs'],]

In [136]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [137]:
te_ary.astype("int")

array([[0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1],
       [0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1],
       [0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0]])

In [138]:
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df, min_support=0.6)

,support,itemsets
0,1.0,(5)
1,0.8,(3)
2,0.6,(10)
3,0.6,(8)
4,0.6,(6)
5,0.8,"(3, 5)"
6,0.6,"(10, 5)"
7,0.6,"(8, 3)"
8,0.6,"(8, 5)"
9,0.6,"(8, 3, 5)"


In [139]:
fpgrowth(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,1.0,(Kidney Beans)
1,0.8,(Eggs)
2,0.6,(Yogurt)
3,0.6,(Onion)
4,0.6,(Milk)
5,0.8,"(Kidney Beans, Eggs)"
6,0.6,"(Kidney Beans, Yogurt)"
7,0.6,"(Onion, Eggs)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Onion, Kidney Beans, Eggs)"


In [140]:
from mlxtend.frequent_patterns import apriori

%timeit -n 100 -r 10 apriori(df, min_support=0.6)

6.66 ms ± 52.2 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [141]:
from mlxtend.frequent_patterns import fpgrowth

%timeit -n 100 -r 10 fpgrowth(df, min_support=0.6)

2.91 ms ± 58.3 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [157]:

body = client_10b8e41eca1446128325a8a246b62cfc.get_object(Bucket='learningassociationsgroceries-donotdelete-pr-bmc4wwzdn1kprb',Key='transactions_by_dept.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,POS Txn,Dept,ID,Sales U
0,16120100160021008773,0261:HOSIERY,250,2
1,16120100160021008773,0634:VITAMINS & HLTH AIDS,102,1
2,16120100160021008773,0879:PET SUPPLIES,158,2
3,16120100160021008773,0973:CANDY,175,2
4,16120100160021008773,0982:SPIRITS,176,1


In [168]:
patterns = pyfpgrowth. find_frequent_patterns(df, 10)

In [190]:
rules = pyfpgrowth. generate_association_rules(patterns,0.8)

In [191]:
def support_count(rhs):
    count = 0
    rhs = set(rhs)
    for j in df["items"]:
        j = set(j)
        if(rhs.issubset(j)):
            count = count+1
    return count

In [211]:
def rules_df(rhs):
    rules = 0
    rhs = set(rhs)
    for i in df["items"]:
        a = support_count(i)
        if(rhs.issubset(i)):
            rhs_support.append(a/len(df))
    return support_count

In [221]:
rules_df["RHS_support"] = rhs_support

TypeError: 'function' object does not support item assignment